In [ ]:
import os
from pathlib import Path
import sys

import polars as pl


sys.path.append(str(Path.cwd().parent))

from libraries.client_stashapp import StashAppClient, get_stashapp_client


stash = get_stashapp_client()
stash_client = StashAppClient()

In [ ]:
oshash_and_duration_match_tag = stash.find_tag({ "name": "Duplicate: OSHASH And Duration Match" })
duration_match_tag = stash.find_tag({ "name": "Duplicate: Duration Match" })
scenes_with_multiple_versions = stash.find_tag({ "name": "Scene: Multiple Versions" })
duration_mismatch_tag = stash.find_tag({ "name": "Duplicate: Duration Mismatch" })

In [ ]:
scenes_with_dupes = stash.find_scenes({
  "file_count": {
    "modifier": "GREATER_THAN",
    "value": 1
  },
  "tags": {
    "value": [],
    "modifier": "INCLUDES",
    "excludes": [scenes_with_multiple_versions["id"]]
  }
}, fragment="id title date studio { name } files { id duration path width height size fingerprints { type value } }")
scenes_with_dupes

In [ ]:
import re


# Function to check if a filename ends with a UUID pattern before the extension
def has_uuid_before_extension(path):
    # Extract the filename from the path
    filename = Path(path).name
    # UUID pattern: 8-4-4-4-12 hexadecimal characters, allowing for a space or dash before it
    uuid_pattern = r"[ -][0-9a-f]{8}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{12}\.[^.]+$"
    return bool(re.search(uuid_pattern, filename.lower()))

scenes_which_need_to_be_updated = []

for scene in scenes_with_dupes:
    if not has_uuid_before_extension(scene["files"][0]["path"]):
        for file in scene["files"][1:]:
            if has_uuid_before_extension(file["path"]):
                scenes_which_need_to_be_updated.append({
                    "scene_id": scene["id"],
                    "file_id": file["id"]
                })
                break

scenes_which_need_to_be_updated

In [ ]:
for row in scenes_which_need_to_be_updated:
    print(row)
    stash.update_scene({
        "id": row["scene_id"],
        "primary_file_id": row["file_id"]
    })